In [1]:
import requests
import torch
from PIL import Image
from io import BytesIO
import os

from diffusers import StableDiffusionImg2ImgPipeline

device = "cuda"
model_id_or_path = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(model_id_or_path, torch_dtype=torch.float16)
pipe = pipe.to(device)

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [2]:
os.chdir("/home/ubuntu/4catalyzer-hackathon")

In [6]:
# Assuming the 'data' folder is in the same directory as your script
data_folder = "./data/dummy/"

# Replace 'sketch-mountains-input.jpg' with the actual filename if needed
image_filename = '1.jpg'

# Build the full path to the image file
image_path = os.path.join(data_folder, image_filename)

# Open and resize the image
init_image = Image.open(image_path).convert("RGB")
# init_image = init_image.resize((768, 512))

prompt = "insert a brain lesion into the MRI image, retaining most of the original MRI image"

images = pipe(prompt=prompt, image=init_image, strength=0.01, guidance_scale=15).images

for idx, image in enumerate(images):
    output_path = f"./opt/{os.path.splitext(os.path.basename(image_filename))[0]}-generated-{idx}-guidance-15-strength-1.jpg"
    image.save(output_path)

0it [00:00, ?it/s]